## See Avalable Microphones

In [1]:
# Using Soundevice
import sounddevice as sd
devices = sd.query_devices()

microphones = [device for device in devices if device['max_input_channels'] > 0]
for idx, microphone in enumerate(microphones):
    print(f"Microphone {microphone['index']}: {microphone['name']} - {microphone['max_input_channels']}")
    
speakers = [device for device in devices if device['max_output_channels'] > 0 and 'NVidia' not in device['name']]
for idx, speaker in enumerate(speakers):
    print(f"Speaker {speaker['index']}: {speaker['name']} - {speaker['max_output_channels']}")

Microphone 7: QHM-990 HD Camera: USB Audio (hw:1,0) - 2
Microphone 16: HD-Audio Generic: ALC1220 Alt Analog (hw:3,2) - 2
Microphone 18: pulse - 32
Microphone 19: default - 32
Speaker 15: HD-Audio Generic: ALC1220 Digital (hw:3,1) - 2
Speaker 17: hdmi - 2
Speaker 18: pulse - 32
Speaker 19: default - 32


In [2]:
# Using PyAudio
import pyaudio
pa = pyaudio.PyAudio()

devices = []
for device_index in range(pa.get_device_count()):
    device_info = pa.get_device_info_by_index(device_index)
    devices.append(device_info)

# print(devices)

microphones = [device for device in devices if device['maxInputChannels'] > 0]
for idx, microphone in enumerate(microphones):
    print(f"Microphone {microphone['index']}: {microphone['name']} InputCh:{microphone['maxInputChannels']} SampleRate:{microphone['defaultSampleRate']} OutCh:{microphone['maxOutputChannels']}")

speakers = [device for device in devices if device['maxOutputChannels'] > 0 and 'NVidia' not in device['name']]
for idx, speaker in enumerate(speakers):
    print(f"Speaker {speaker['index']}: {speaker['name']} InputCh:{speaker['maxInputChannels']} SampleRate:{speaker['defaultSampleRate']} OutCh:{speaker['maxOutputChannels']}")

Microphone 7: QHM-990 HD Camera: USB Audio (hw:1,0) InputCh:2 SampleRate:16000.0 OutCh:0
Microphone 16: HD-Audio Generic: ALC1220 Alt Analog (hw:3,2) InputCh:2 SampleRate:44100.0 OutCh:0
Microphone 18: pulse InputCh:32 SampleRate:44100.0 OutCh:32
Microphone 19: default InputCh:32 SampleRate:44100.0 OutCh:32
Speaker 15: HD-Audio Generic: ALC1220 Digital (hw:3,1) InputCh:0 SampleRate:44100.0 OutCh:2
Speaker 17: hdmi InputCh:0 SampleRate:44100.0 OutCh:2
Speaker 18: pulse InputCh:32 SampleRate:44100.0 OutCh:32
Speaker 19: default InputCh:32 SampleRate:44100.0 OutCh:32


## Record Audio

In [3]:
import ipywidgets as widgets
import threading
Thread = threading.Thread
import pyaudio

class Recorder:
    def __init__(self, 
                 microphone_index: int, 
                #  speaker_index: int, 
                #  output: bool = False
                ) -> None:
        
        # Start PyAudio Stream
        self.pa = pyaudio.PyAudio()
        
        self.AUDIO_FORMAT = pyaudio.paFloat32
        self.CHUNK_LENGTH = 1024
        # self.MAX_RECORD_SECONDS = 5
        self.SAMPLE_RATE = int(
            self.pa.get_device_info_by_index(microphone_index)['defaultSampleRate']
                )
        # self.MIC_IN_CHANNELS = int(
        #     self.pa.get_device_info_by_index(microphone_index)['maxInputChannels']
        #         )
        self.MIC_IN_CHANNELS = 1
        self.MIC_NAME = self.pa.get_device_info_by_index(microphone_index)['name']
        
        self.output = widgets.Output()
        
        self.audio_frames = []
    
        self.stream = self.pa.open(
            rate = self.SAMPLE_RATE,
            channels = self.MIC_IN_CHANNELS,
            format = self.AUDIO_FORMAT,
            input = True,
            input_device_index = microphone_index,
            # output = output,
            # output_device_index = speaker_index,
            frames_per_buffer = self.CHUNK_LENGTH,
        )
        
        return None
    
    def _print(self, *args):
        with self.output:
            print(*args)

    def start_recording(self):
        self._print('Recording using: ',self.MIC_NAME)
        self._print('Sample Rate: ', self.SAMPLE_RATE)
        
        self.stream.start_stream() if self.stream.is_stopped() else self._print('Stream already started')
        self.audio_frames = []  # Clear previous frames
        
        while self.stream.is_active():
            data = self.stream.read(self.CHUNK_LENGTH, exception_on_overflow = False)
            self.audio_frames.append(data)
            # self.stream.write(data, self.CHUNK_LENGTH, exception_on_underflow = False)
    
    def stop_recording(self):
        self.stream.stop_stream()
        self._print('Stopped Recording')
        
        # Stop the thread
        for thread in threading.enumerate():
            if thread.name.strip() == 'recording_thread':
                thread.join()
                self._print('Recording thread stopped')
                break

In [4]:
record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

recorder = Recorder(18)

record_button.on_click(lambda btn: Thread(target=recorder.start_recording, name='recording_thread').start() if not recorder.stream.is_active() else print('Cannot start recording, stream already active'))
stop_button.on_click(lambda btn: Thread(target=recorder.stop_recording, name='stop_recording_thread').start())

from IPython.display import display
display(record_button, stop_button, recorder.output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

Cannot start recording, stream already active
Cannot start recording, stream already active
Cannot start recording, stream already active
Cannot start recording, stream already active
Cannot start recording, stream already active


In [5]:
# See all threads running
for thread in threading.enumerate():
    print(thread.name)

MainThread
IOPub
Heartbeat
Thread-2 (_watch_pipe_fd)
Thread-3 (_watch_pipe_fd)
Control
IPythonHistorySavingThread


## Play Audio

In [7]:
# Play the audio
import numpy as np
from IPython.display import Audio
audio = np.frombuffer(b''.join(recorder.audio_frames), dtype=np.float32)
Audio(audio, rate=recorder.SAMPLE_RATE)